# Agglomerative Clustering

### 01_06_20

## Importing Libraries

In [42]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)

#from pandas import DataFrame, read_csv
import pandas as pd

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
#import scipy.signal as signal
from scipy.signal import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics

import sys  
#sys.path.insert(0, '/Users/louiseplacidet/Desktop/PIR/GITPIR/GIT_29_04/PIR/AdabandFlt')
#sys.path.insert(0, '/Users/SYL21/External_Drive/SUPAERO/PIR/AdabandFlt')
sys.path.insert(0, '/Users/louiseplacidet/Desktop/PIR/GITPIR/GIT_29_04/PIR/AdabandFlt')
#from AdaBandFlt import *
#from V2AdaBandFlt import *
from V3AdaBandFlt import *

%matplotlib tk

## Loading Data

In [9]:
#load data
# file path of csv file
#Location = r'/Users/SYL21/External_Drive/SUPAERO/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/External_Drive/SUPAERO/PIR/Data/Wetransfer_data/E18KABaseline_BcutV2groundAll.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'
Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_new_spike_data/539W6cbaseRaw.txt'

# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## Filtering the signal

### Global Parameters

In [22]:
size_of_data = 1000000
fs = 50000

method_align = 'indice_1er_depass'
time_before = 0.0015
time_after = 0.0015

threshold_factor = 3.5
maxseparation = 0.0008

#n_electrode = 4

### Filtering functions and cutoff frequencies

In [11]:
lowcut = 100.0
highcut = 5000.0

In [12]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [13]:
signal_to_filter = df.iloc[:size_of_data,6] #Electrode 58

y = butter_bandpass_filter(signal_to_filter, lowcut, highcut, fs, order=6)

filtereddf = pd.DataFrame(y)
filtereddf.index = df.index

filtereddf

,0
%t,
0.00,0.898347
0.02,-1.423672
0.04,-3.482439
0.06,-5.068991
0.08,-6.071031
...,...
299999.90,-5.381811
299999.92,-4.461242
299999.94,-3.219172


In [14]:
signal_filtered = filtereddf.iloc[:,0]
signal = filtereddf.iloc[:,0] ##selecting an electrode to use

## Detecting and Aligning the Spikes

### Setting up the Thresholds adapted to Noise Levels

In [15]:
noise_levels = init_noise_levels(signal_filtered, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25)

#plt.figure()
#plt.plot(noise_levels)
#plt.grid(True)
#plt.xlabel('Time')
#plt.ylabel('Noise Amplitude [µV]')
#plt.title('Noise Levels')

### Detecting Spikes

In [23]:
spike_info = find_spikes(signal, noise_levels, fs, 
                           window_size = 0.002, 
                           noise_window_size = 0.01,
                           threshold_factor = threshold_factor,
                           maxseparation = maxseparation)  
                
#spike_info.columns = ['indice_max','indice_min','indice_depass_positif','indice_depass_negatif', 'indice_1er_depass','indice_zero_central','i_max-i_min','Delta_amplitudes']
spike_info

,indice_min,indice_1er_depass,indice_max_gauche,indice_max_droite,Delta_amplitudes
0,48001,48001,47964,nan,20.420455
1,51265,51264,51253,nan,16.810932
2,127483,127482,127465,nan,17.072964
3,176846,176845,nan,176880,19.056821
4,183942,183941,nan,183969,17.201650
...,...,...,...,...,...
373,14770249,14770246,14770232,nan,19.070289
374,14864191,14864189,14864177,nan,18.929579
375,14897921,14897920,14897884,nan,16.825488
376,14925034,14925032,nan,14925068,19.215302


### Recording the spikes

In [28]:
spike_data = record_spikes(signal, fs, spike_info,
                           method_align,
                           t_before = time_before, 
                           t_after = time_after)

(379, 151)


In [29]:
spike_data_oneline = record_spikes_oneline(signal, fs, spike_info,
                                           method_align,
                                           t_before = time_before,
                                           t_after = time_after)


### Plotting the spikes

In [30]:
plt.plot(df.index, signal, color = 'blue')
plt.plot(spike_data_oneline.index, spike_data_oneline, color = 'red')
plt.title('Filtered Signal with Detected Spikes with RMS')
plt.xlabel('Time Windows')
plt.ylabel('Amplitude [µV]')
plt.grid(True)

In [31]:
print_spikes(spike_data,
             t_before_alignement = time_before,
             first_spike = 1,
             last_spike = 50,
             fs = fs)

# Bilan PCA + AGGLOMERATIVE CLUSTERING

## PCA and AGGLOMERATIVE CLUSERING on spikes

In [32]:
def PCA_and_AGGLOCLUST_spikes(spike_data, spike_info, nb_PCA_components=3,
                              n_clusters=5, metric='euclidean', linkage='ward'):
    
    ## on rééquilibre les valeurs dans les différentes dimensions
    #pca_data = np.array(spike_data.iloc[:,1:].values).transpose()
    #pca_data = StandardScaler().fit_transform(pca_data) # normalizing the features
    
    ## PCA
    pca_data = np.array(spike_data.iloc[:,1:].values).transpose()
    pca = PCA(n_components=nb_PCA_components)
    pca.fit(pca_data)
    PCA_X = pca.transform(pca_data)
    
    ## AGGLOMERATIVE CLUSTERING
    ## Different linkages: 'ward', 'average', 'complete', 'single'
    
    aggloclustering = AgglomerativeClustering(n_clusters=n_clusters, affinity = metric,
                                    linkage=linkage)
    aggloclustering.fit(PCA_X)
    
    labels = aggloclustering.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    
    ## Ajout du label des clusters dans spike info
    spike_info['cluster_label'] = aggloclustering.labels_
    
    return PCA_X, aggloclustering, spike_info
    

#### Clustering function info: 
AgglomerativeClustering(n_clusters=2, *, affinity='euclidean', memory=None, connectivity=None, compute_full_tree='auto', linkage='ward', distance_threshold=None)

- affinity: “euclidean”, “l1”, “l2”, “manhattan”, “cosine”, or “precomputed”
    NB: if linkage is ward: only "euclidean" is accepted
- linkage{“ward”, “complete”, “average”, “single”}, default=”ward”

## Plotting the PCA

In [33]:
## Fonction qui plot la PCA

def PCA_plot(PCA_X):
    
    fig = plt.figure(4,figsize=(4,3))
    plt.clf()
    ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
    plt.cla()
  
    ax.scatter(PCA_X[:, 0], PCA_X[:, 1], PCA_X[:, 2], cmap=plt.cm.nipy_spectral,
           edgecolor='k')

    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])

    plt.show()

## Plotting the AGGLO CLUSTERING

In [34]:
## Fonction qui plot les clusters d'OPTICS

def print_clusters_3d(labels, PCA_X):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    nb_clusters = max(labels) + 1

    a = [i for i in range(len(labels))]
    b = np.transpose([a,list(labels)])

    for nb in range(nb_clusters):
        legend = 'Cluster n°'+str(nb)
        data = PCA_X[[x for x,y in b if y==nb],:]
        ax.scatter(data[:,0], data[:,1], data[:,2],label=legend)
    data = PCA_X[[x for x,y in b if y==-1],:]
    ax.scatter(data[:,0], data[:,1], data[:,2], c='black',label='Noise cluster')    
    
    ax.set_title('Nombre de cluster(s) :' + str(nb_clusters))
    ax.legend()
    plt.show()
    
#print_clusters_3d(kmeans.labels_, X)

## Plotting the spikes from the different clusters after OPTICS

In [35]:
def print_spikes_clusterized(spike_data,
                             labels,
                             t_before_alignement = 0,
                             nb_spike = 20,
                             y_lim_min = -50,
                             y_lim_max = 60,
                             fs = 25000):
    
    nb_clusters = max(labels) + 1
        
    if (-1 in labels) == True:
        nb_clusters_ = nb_clusters + 1
    else:
        nb_clusters_ = nb_clusters
    
    nb_line = nb_clusters_//2
    if nb_clusters_%2 != 0:
        nb_line += 1
    
    #spike_data.iloc[:,first_spike:last_spike].plot()
    t_b = int(np.round(fs*(t_before_alignement)))
    y = (spike_data.iloc[:,0]-t_b)*1000/fs
        
    a = [i+1 for i in range(len(labels))]
    b = np.transpose([a,list(labels)])
    
    figure = plt.figure()
    plt.gcf().subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.5)
    for nb in range(nb_clusters):
        data = spike_data.iloc[:,[x for x,y in b if y==nb]]
        m = len(data.values[0])
        nb_spikes_in_cluster = len(data.columns)

        
        kept = []
        
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = data.iloc[:,kept].values
        axes = figure.add_subplot(nb_line, 2, nb+1)
        axes.plot(y, x)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster numero ' + str(nb))
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid(True)
        axes.legend([str(nb_spikes_in_cluster)+' spikes'])
        
    if (-1 in labels) == True:
        data = spike_data.iloc[:,[x for x,y in b if y==-1]]
        m = len(data.values[0])
        nb_spikes_in_cluster = len(data.columns)
    
        kept = []
        
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = data.iloc[:,kept].values
        
        axes = figure.add_subplot(nb_line, 2, nb+2)
        axes.plot(y, x)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster de bruit')
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid(True)
        axes.legend([str(nb_spikes_in_cluster)+' spikes'])
        
#print_spikes_clusterized(spike_data,
#                             dbscan.labels_,
#                             t_before_alignement = 0.0015,
#                             nb_spike = 20,
#                             y_lim_min = -50,
#                             y_lim_max = 60,
#                             fs = 25000)

## Printing the spikes from clusters on original signal

In [36]:
## Fonction qui affiche les spikes des différents clusters sur échelle temporelle

def print_spikes_clusterized_oneline(signal,
                                     updated_spike_infos,
                                     align_method = 'indice_zero_central',
                                     t_before = 0.001,
                                     t_after = 0.002,
                                     fs = 25000,
                                     y_lim_min = -50,
                                     y_lim_max = 60,
                                     separate_plot = False):
    
    labels = updated_spike_infos['cluster_label'].values
    nb_clusters = max(labels) + 1
    
    if (-1 in labels) == True:
        nb_clusters_ = nb_clusters + 2
    else:
        nb_clusters_ = nb_clusters + 1
    
    nb_line = nb_clusters_//2
    if nb_clusters_%2 != 0:
        nb_line += 1
    
    ghost_array = np.array(['NaN' for x in range(len(signal))])
    ghost_array = ghost_array.astype(float)
    spike_data_clusterized_oneline = []
    legend = ['Signald\'origine']
    
    figure = plt.figure()
    plt.gcf().subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.5)
    for nb in range(nb_clusters):
        
        spike_data_oneline = record_spikes_oneline(signal,
                              fs,
                              updated_spike_infos.loc[updated_spike_infos['cluster_label'] == nb],
                              align_method,
                              t_before = t_before,
                              t_after = t_after)
        spike_data_oneline = np.resize(spike_data_oneline.values,len(spike_data_oneline.values))
        spike_data_clusterized_oneline.append(spike_data_oneline)
        
        
        axes = figure.add_subplot(nb_line, 2, nb+1)
        axes.plot(signal.index, signal.values)
        for ghost in range(nb):
            axes.plot(signal.index, ghost_array)
        axes.plot(signal.index, spike_data_oneline)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster n°' + str(nb))
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid()
        
        legend.append('Cluster n°' + str(nb))
        
    if (-1 in labels) == True:
        spike_data_oneline = record_spikes_oneline(signal,
                              fs,
                              updated_spike_infos.loc[updated_spike_infos['cluster_label'] == -1],
                              align_method,
                              t_before = t_before,
                              t_after = t_after)
        spike_data_oneline = np.resize(spike_data_oneline.values,len(spike_data_oneline.values))
        spike_data_clusterized_oneline.append(spike_data_oneline)
        
        
        axes = figure.add_subplot(nb_line, 2, nb+2)
        axes.plot(signal.index, signal.values)
        axes.plot(signal.index, spike_data_oneline)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster de bruit')
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid()
                
        legend.append('Cluster de bruit')
        
        axes = figure.add_subplot(nb_line, 2, nb+3)
    else:
        axes = figure.add_subplot(nb_line, 2, nb+2)
    
    axes.plot(signal.index, signal.values)
    axes.plot(signal.index, np.transpose(spike_data_clusterized_oneline))
    axes.legend(legend)
    axes.set_xlabel('Time in ms')
    axes.set_title('Tous les clusters')
    axes.set_ylim(y_lim_min , y_lim_max)
    axes.grid()
        
        
        

#print_spikes_clusterized_oneline(signal,
#                                 updated_spike_infos,
#                                 align_method = 'indice_zero_central',
#                                 t_before = 0.001,
#                                 t_after = 0.002,
#                                 fs = 25000,)


In [37]:
## Fonction qui affiche les spikes des différents clusters sur échelle temporelle

def print_spikes_clusterized_oneline_total(signal,
                                     updated_spike_infos,
                                     align_method = 'indice_zero_central',
                                     t_before = 0.001,
                                     t_after = 0.002,
                                     fs = 25000,
                                     y_lim_min = -50,
                                     y_lim_max = 60,
                                     separate_plot = False):
    
    labels = updated_spike_infos['cluster_label'].values
    nb_clusters = max(labels) + 1
    
    if (-1 in labels) == True:
        nb_clusters_ = nb_clusters + 2
    else:
        nb_clusters_ = nb_clusters + 1
    
    nb_line = nb_clusters_//2
    if nb_clusters_%2 != 0:
        nb_line += 1
    
    ghost_array = np.array(['NaN' for x in range(len(signal))])
    ghost_array = ghost_array.astype(float)
    spike_data_clusterized_oneline = []
    legend = ['Signald\'origine']
    
    figure = plt.figure()
    plt.gcf().subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.5)
    for nb in range(nb_clusters):
        
        spike_data_oneline = record_spikes_oneline(signal,
                              fs,
                              updated_spike_infos.loc[updated_spike_infos['cluster_label'] == nb],
                              align_method,
                              t_before = t_before,
                              t_after = t_after)
        spike_data_oneline = np.resize(spike_data_oneline.values,len(spike_data_oneline.values))
        spike_data_clusterized_oneline.append(spike_data_oneline)
        
        legend.append('Cluster n°' + str(nb))

        
    if (-1 in labels) == True:
        spike_data_oneline = record_spikes_oneline(signal,
                              fs,
                              updated_spike_infos.loc[updated_spike_infos['cluster_label'] == -1],
                              align_method,
                              t_before = t_before,
                              t_after = t_after)
        spike_data_oneline = np.resize(spike_data_oneline.values,len(spike_data_oneline.values))
        spike_data_clusterized_oneline.append(spike_data_oneline)
        legend.append('Cluster de bruit')
    
    axes = figure.add_subplot(1, 1, 1)
    axes.plot(signal.index, signal.values)
    axes.plot(signal.index, np.transpose(spike_data_clusterized_oneline))
    axes.legend(legend)
    axes.set_xlabel('Time in ms')
    axes.set_title('Tous les clusters')
    axes.set_ylim(y_lim_min , y_lim_max)
    axes.grid()
        
        
        
        

#print_spikes_clusterized_oneline_total(signal,
#                                 updated_spike_infos,
#                                 align_method = 'indice_zero_central',
#                                 t_before = 0.001,
#                                 t_after = 0.002,
#                                 fs = 25000,)

## Histogram of Amplitudes and Time

In [38]:
## Histogramme de différences d'amplitude Peak-to-Peak des spikes

def histogram_spikes_amplitude(spike_info):
    plt.hist(spike_info['Delta_amplitudes'], bins='auto')
    plt.title("Histogram of Spike Peak-to-Peak Amplitude")
    plt.show()

In [39]:
## Histogramme de l'écartement temporel des spikes (imax-imin)

def histogram_spikes_time(spike_info):
    plt.hist(spike_info['i_max-i_min'], bins='auto')
    plt.title("Histogram of Spike i_max-i_min")
    plt.show()

In [40]:
## Histogramme des différences d'amplitude Peak-to-Peak des spikes:

## ¡¡¡ATTENTION!!! Il faut que le spike info en entrée, soit mis à jour avec les labels des clusters

def histogram_amplitude_clusterized(updated_spike_info):
    
    labels = updated_spike_info['cluster_label'].values
    nb_clusters = max(labels) + 1
    
    for nb in range(nb_clusters):
        local_info = spike_info.loc[spike_info['cluster_label'] == nb]
        plt.figure()
        plt.hist(local_info['cluster_label'], bins='auto')
        plt.title("Histogram of Spike i_max-i_min of cluster n°"+str(nb))
        plt.show()
    
    if(-1 in labels) == True:
        local_info = spike_info.loc[spike_info['cluster_label']==-1]
        plt.figure()
        plt.hist(local_info['cluster_label'], bins='auto')
        plt.title("Histogram of Spike i_max-i_min of cluster of noise")
        plt.show()

## Tests des fonctions

In [43]:
## Function: PCA_and_AGGLOCLUST_spikes(spike_data, spike_info, nb_PCA_components=3,
                                        #n_clusters=5,metric = 'euclidean',linkage='ward')

PCA_X, aggloclustering, updated_spike_info = PCA_and_AGGLOCLUST_spikes(spike_data,
                                                                       spike_info, nb_PCA_components=3,
                                                                       n_clusters=5, metric="euclidean", 
                                                                       linkage="ward")

In [44]:
aggloclustering.labels_;

In [45]:
# Function: PCA_plot(spike_data, nb_clusters=3)

PCA_plot(PCA_X)

In [46]:
# Function: print_clusters_3d(labels, PCA_X)

print_clusters_3d(aggloclustering.labels_, PCA_X)

In [47]:
# Function: print_spikes_clusterized(spike_data,
#                             labels,
#                             t_before_alignement = 0.0015,
#                             nb_spike = 20,
#                             y_lim_min = -50,
#                             y_lim_max = 60,
#                             fs = 25000)
                        
print_spikes_clusterized(spike_data,
                             aggloclustering.labels_,
                             t_before_alignement = time_before,
                             nb_spike = 20,
                             y_lim_min = -50,
                             y_lim_max = 60,
                             fs = fs)                        

In [48]:
# Function: print_spikes_clusterized_oneline(signal,
#                                 updated_spike_infos,
#                                 align_method = 'indice_zero_central',
#                                 t_before = 0.001,
#                                 t_after = 0.002,
#                                 fs = 25000,)

print_spikes_clusterized_oneline(signal,
                                 updated_spike_info,
                                 align_method = method_align,
                                 t_before = time_before,
                                 t_after = time_after,
                                 fs = fs,)


/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [50]:
# Function: print_spikes_clusterized_oneline_total(signal,
#                                 updated_spike_infos,
#                                 align_method = 'indice_zero_central',
#                                 t_before = 0.001,
#                                 t_after = 0.002,
#                                 fs = 25000,)

print_spikes_clusterized_oneline_total(signal,
                                 updated_spike_info,
                                 align_method = method_align,
                                 t_before = time_before,
                                 t_after = time_after,
                                 fs = fs,)


/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)


In [33]:
# Function: histogram_spikes_amplitude(spike_info)

histogram_spikes_amplitude(spike_info)

In [30]:
# Function: histogram_spikes_time(spike_info)

histogram_spikes_time(spike_info)

/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [36]:
# Function: histogram_amplitude_clusterized(spike_info)

histogram_amplitude_clusterized(updated_spike_info)